In [1]:
import pandas as pd 
import os 
import shutil

In [5]:
ds = pd.read_excel("release_midas.xlsx")

In [60]:
ds.head(20)

,Unnamed: 0,midas_record_id,midas_file_name,midas_iscontrol,midas_distance,midas_location,midas_path,midas_pathreport,midas_gender,midas_age,midas_fitzpatrick,midas_melanoma,midas_ethnicity,midas_race,clinical_impression_1,clinical_impression_2,clinical_impression_3,length_(mm),width_(mm),predicted_category
0,0,1,s-prd-398966407.jpg,no,1ft,chest,malignant- sccis,"focal squamous cell carcinoma, in situ, aris...",female,80,"ii fair skin, blue eyes",no,no,white,7-malignant-bcc,6-benign-other,1-benign-melanocytic nevus,8.0,5.0,SCC
1,1,1,s-prd-398966642.jpg,no,6in,chest,malignant- sccis,"focal squamous cell carcinoma, in situ, aris...",female,80,"ii fair skin, blue eyes",no,no,white,7-malignant-bcc,6-benign-other,1-benign-melanocytic nevus,8.0,5.0,SCC
2,2,1,s-prd-398966845.jpg,no,dscope,chest,malignant- sccis,"focal squamous cell carcinoma, in situ, aris...",female,80,"ii fair skin, blue eyes",no,no,white,7-malignant-bcc,6-benign-other,1-benign-melanocytic nevus,8.0,5.0,SCC
3,3,1,s-prd-398967381.jpg,no,1ft,l lower back,benign-melanocytic nevus,"intradermal melanocytic nevus, transected at t...",female,80,"ii fair skin, blue eyes",no,no,white,1-benign-melanocytic nevus,6-benign-other,6-benign-other,6.0,5.0,Benign_or_Other
4,4,1,s-prd-398967587.jpg,no,dscope,l lower back,benign-melanocytic nevus,"intradermal melanocytic nevus, transected at t...",female,80,"ii fair skin, blue eyes",no,no,white,1-benign-melanocytic nevus,6-benign-other,6-benign-other,6.0,5.0,Benign_or_Other
5,5,1,s-prd-398968016.jpg,no,1ft,left upper buttock,benign-melanocytic nevus,"intradermal melanocytic nevus, transected at t...",female,80,"ii fair skin, blue eyes",no,no,white,1-benign-melanocytic nevus,6-benign-other,NaN,5.0,3.0,Benign_or_Other
6,6,1,s-prd-398968141.jpg,no,6in,left upper buttock,benign-melanocytic nevus,"intradermal melanocytic nevus, transected at t...",female,80,"ii fair skin, blue eyes",no,no,white,1-benign-melanocytic nevus,6-benign-other,NaN,5.0,3.0,Benign_or_Other
7,7,2,s-prd-399001840.jpg,no,6in,right upper eyelid,benign-other,findings consistent with ruptured follicle/e...,male,34,"ii fair skin, blue eyes",no,no,white,6-benign-other,6-benign-other,NaN,4.0,2.0,Benign_or_Other
8,8,2,s-prd-399001676.jpg,no,1ft,right upper eyelid,benign-other,findings consistent with ruptured follicle/e...,male,34,"ii fair skin, blue eyes",no,no,white,6-benign-other,6-benign-other,NaN,4.0,2.0,Benign_or_Other
9,9,2,s-prd-399002046.jpg,no,dscope,right upper eyelid,benign-other,findings consistent with ruptured follicle/e...,male,34,"ii fair skin, blue eyes",no,no,white,6-benign-other,6-benign-other,NaN,4.0,2.0,Benign_or_Other


In [59]:
ds.describe()

,Unnamed: 0,midas_record_id,midas_age,length_(mm),width_(mm)
count,3416.000000,3416.000000,3416.000000,3399.000000,3399.000000
mean,1707.500000,355.321136,63.024883,7.122065,5.870550
std,986.258587,234.059280,16.292844,5.225501,4.591287
min,0.000000,1.000000,21.000000,0.000000,0.000000
25%,853.750000,152.000000,53.000000,4.000000,4.000000
50%,1707.500000,327.000000,66.000000,6.000000,5.000000
75%,2561.250000,552.000000,75.000000,9.000000,7.000000
max,3415.000000,815.000000,99.000000,80.000000,80.000000


In [58]:
print(ds['midas_path'].unique())
print("Value counts:")
print(ds['midas_path'].value_counts())

['malignant- sccis' 'benign-melanocytic nevus' 'benign-other' nan
 'malignant- scc' 'malignant- ak' 'benign-seborrheic keratosis'
 'malignant- bcc' 'malignant- other'
 'other- melanocytic lesion, possible re-excision (severe, spitz, aimp)'
 'other- non-neoplastic, inflammatory, infectious' 'malignant- melanoma'
 'benign-hemangioma' 'benign-dermatofibroma'
 'melanocytic tumor, possible re-excision (severe, spitz, aimp)' 0
 'benign-fibrous papule']
Value counts:
midas_path
malignant- bcc                                                           608
benign-melanocytic nevus                                                 578
benign-other                                                             421
benign-seborrheic keratosis                                              242
malignant- melanoma                                                      238
malignant- scc                                                           203
malignant- ak                                                 

In [ ]:
# make mapping for melanoma, BCC, SCC, Actinic Keratosis, and Merkel Cell also benign 

In [40]:
cancer_map = {
    "Melanoma": ["melanoma", "lentigo"],
    "BCC": ["bcc"],
    "SCC": ["scc", "sccis"],  
    "Actinic Keratosis": ["ak"],
    "Merkel Cell": ["merkel"]
}

In [45]:
source_images = "Images"
output_images = "Sorted_Images"

In [18]:
os.makedirs(os.path.join(output_images, "Benign_or_Other"), exist_ok=True)

In [11]:
for class_name in cancer_map.keys():
    os.makedirs(os.path.join(output_images, class_name), exist_ok=True)

In [46]:
counters = {k: 0 for k in cancer_map.keys()}
counters["Benign_or_Other"] = 0 

In [47]:
for index, row in ds.iterrows():
    filename = row["midas_file_name"] #images
    pathology_text = str(row["midas_path"]).lower()
    destination = "Benign_or_Other"

    for cancer_type, keywords in cancer_map.items(): # check if any keywords appear 
        if any(keyword in pathology_text for keyword in keywords):
            destination = cancer_type
            break 
    
    src = os.path.join(source_images, filename)
    dst = os.path.join(output_images, destination, filename)

    try:
        if os.path.exists(src):
            shutil.copy(src, dst)
            counters[destination] += 1
        else:
            pass
    except Exception as e:
        print(F"error moving {filename}: {e}")

for category, count in counters.items():
    print(f"{category}: {count} images")

Melanoma: 234 images
BCC: 594 images
SCC: 365 images
Actinic Keratosis: 176 images
Merkel Cell: 0 images
Benign_or_Other: 1786 images


3155 total images, pretty imbalanced with a lot in benign or other, no merkel 